In [ ]:
!gdown 1Ia9c5HUQL_-DtG2ZfoIqA-_FjYYSiwaU

Downloading...
From (original): https://drive.google.com/uc?id=1Ia9c5HUQL_-DtG2ZfoIqA-_FjYYSiwaU
From (redirected): https://drive.google.com/uc?id=1Ia9c5HUQL_-DtG2ZfoIqA-_FjYYSiwaU&confirm=t&uuid=85f98c90-9cea-4cd9-8223-4bfd2b37ec8a
To: /content/trinetx_eGFR_gt20_95KPatients.zip
100% 3.75G/3.75G [01:23<00:00, 45.2MB/s]


In [ ]:
!unzip trinetx_eGFR_gt20_95KPatients.zip

Archive:  trinetx_eGFR_gt20_95KPatients.zip
  inflating: patient.csv             
  inflating: diagnosis.csv           
  inflating: procedure.csv           
  inflating: genomic.csv             
  inflating: medication_ingredient.csv  
  inflating: lab_result.csv          
  inflating: tumor_properties.csv    
  inflating: oncology_treatment.csv  
  inflating: tumor.csv               
  inflating: vitals_signs.csv        
  inflating: cohort_details.csv      
  inflating: dataset_details.csv     
  inflating: patient_cohort.csv      
  inflating: chemo_lines.csv         
  inflating: encounter.csv           
  inflating: medication_drug.csv     
  inflating: standardized_terminology.csv  
  inflating: manifest.csv            
  inflating: readme.txt              
  inflating: datadictionary.xlsx     
  inflating: datadictionary.pdf      
  inflating: FAQ.pdf                 


In [ ]:
#ICD Codes for AKI:
code_system = 'ICD-10-CM'
aki_codes = ["A98.5", "D59.3", "K76.7", "N00.8", "N00.9", "N01.3", "N04.2", "N04.7", "N17", "N17.0", "N17.1", "N17.9", "N28.9", "O90.4", "S37.0"]

In [ ]:
import pandas as pd
diagnosis = pd.read_csv('diagnosis.csv')

In [ ]:
#aki patient
aki_patients = diagnosis[(diagnosis['code_system'] == 'ICD-10-CM') & (diagnosis['code'].isin(aki_codes))]

In [ ]:
aki_patients.patient_id.nunique()

32090

In [ ]:
aki_first_dates = aki_patients.loc[aki_patients.groupby('patient_id')['date'].idxmin()]
cols = ['patient_id', 'date']
aki_first_dates_dates = aki_first_dates[cols]

In [ ]:
dialysis_codes1 = {'CPT': ['90935', '1012752', '90999', '90937', '90947', '90945']}
dialysis_codes2 = {'ICD-10-CM': ['I12.0', 'N18.6', 'Z99.2', 'D63.1']}
dialysis_codes3 = {'HCPCS': ['G0257']}

procedure = pd.read_csv('procedure.csv')

In [ ]:
procedure_HCPCS_dialysis = procedure[(procedure.code_system=='HCPCS') & (procedure.code.isin(dialysis_codes3.get('HCPCS')))]

In [ ]:
procedure_CPT_dialysis = procedure[(procedure.code_system=='CPT') & (procedure.code.isin(dialysis_codes1.get('CPT')))]

In [ ]:
diagnosis_ICD_dialysis = diagnosis[(diagnosis.code_system=='ICD-10-CM') & (diagnosis.code.isin(dialysis_codes2.get('ICD-10-CM')))]

In [ ]:
aaaaa = pd.concat([procedure_HCPCS_dialysis[cols], procedure_CPT_dialysis[cols], diagnosis_ICD_dialysis[cols]])
procedure_dialysis_dates = aaaaa.groupby('patient_id')['date'].min()
merged_patients = pd.merge(aki_first_dates_dates, procedure_dialysis_dates, on='patient_id', how='inner')
merged_patients['date_x'] = pd.to_datetime(merged_patients['date_x'], format='%Y%m%d') #aki diagnosis date
merged_patients['date_y'] = pd.to_datetime(merged_patients['date_y'], format='%Y%m%d') #dialysis date
merged_patients['day_gap'] = (merged_patients['date_x'] - merged_patients['date_y']).dt.days #aki patients who need dialysis: target
dialysis_patients = merged_patients

In [ ]:
dialysis_patients.patient_id.nunique()

9475

In [ ]:
drugs = pd.read_csv(
    'medication_drug.csv',
    low_memory=False,
    dtype={'code': str}
)

In [ ]:
drugs.drop(columns=['derived_by_TriNetX', 'source_id'], inplace=True)

In [ ]:
aki_drugs = drugs.loc[drugs['patient_id'].isin(aki_patients['patient_id'])] #drugs used by aki patients

In [ ]:
len(aki_drugs)

25854870

In [ ]:
dialysis_patient_drugs = pd.merge(dialysis_patients, aki_drugs, on='patient_id', how='inner')

dialysis_patient_drugs['drug_date'] = pd.to_datetime(
    dialysis_patient_drugs['start_date'],
    format='%Y%m%d',
    errors='coerce'
)

dialysis_patient_between_0_1100 = dialysis_patients[(dialysis_patients.day_gap<=0) & (dialysis_patients.day_gap>=-1100)]

dialysis_patient_drugs_in_range = dialysis_patient_drugs[(dialysis_patient_drugs.drug_date>=dialysis_patient_drugs.date_x) &
 (dialysis_patient_drugs.drug_date<=dialysis_patient_drugs.date_y) & (dialysis_patient_drugs.day_gap<=0) & (dialysis_patient_drugs.day_gap>=-1100)]

In [ ]:
dialysis_patient_drugs_in_range.patient_id.nunique()

4716

In [ ]:
aki_drugs.head()

,patient_id,encounter_id,unique_id,code_system,code,start_date,route,brand,strength,quantity_dispensed,days_supply
640,HRC,KRbC,HhBU,RxNorm,861522,20090410,Injectable Product,Demerol,Unknown,NaN,NaN
641,HRC,KhZC,HxHU,RxNorm,1791409,20100508,Injectable Product,Osmitrol,Unknown,NaN,NaN
642,HRC,KhZC,HhLU,RxNorm,200251,20100512,Oral Product,Unknown,Unknown,NaN,NaN
643,HRC,KhZC,HxLU,RxNorm,200251,20100513,Oral Product,Unknown,Unknown,NaN,NaN
644,HRC,KBZC,HBMU,RxNorm,200251,20100516,Oral Product,Unknown,Unknown,NaN,NaN


In [ ]:
import numpy as np
no_dialysis_patient = aki_first_dates_dates[~aki_first_dates_dates.patient_id.isin(dialysis_patients.patient_id)] #aki patients who don't need dialysis
no_dialysis_patient_drugs = no_dialysis_patient.merge(aki_drugs, on='patient_id', how='inner')
mean = -dialysis_patient_between_0_1100.day_gap.mean()
std = dialysis_patient_between_0_1100.day_gap.std()
no_dialysis_patient_drugs['day_gap'] = np.random.normal(loc=mean, scale=std, size=len(no_dialysis_patient_drugs)) # create artificial gap for non-target patients

no_dialysis_patient_drugs.rename(columns={'date': 'diagnosis_date'}, inplace=True)

no_dialysis_patient_drugs['diagnosis_date'] = pd.to_datetime(
    no_dialysis_patient_drugs['diagnosis_date'],
    format='%Y%m%d',
    errors='coerce'
)

no_dialysis_patient_drugs['drug_date'] = pd.to_datetime(
    no_dialysis_patient_drugs['start_date'],
    format='%Y%m%d',
    errors='coerce'
)

day_deltas = pd.to_timedelta(
    no_dialysis_patient_drugs['day_gap'].astype(int),
    unit='D'
)

# vectorized addition
no_dialysis_patient_drugs['end_date'] = (
    no_dialysis_patient_drugs['diagnosis_date'] + day_deltas
)

no_dialysis_patient_drugs_in_range = no_dialysis_patient_drugs[(no_dialysis_patient_drugs.drug_date>=no_dialysis_patient_drugs.diagnosis_date)
& (no_dialysis_patient_drugs.drug_date<=no_dialysis_patient_drugs.end_date) & (no_dialysis_patient_drugs.day_gap>=0)
& (no_dialysis_patient_drugs.day_gap<=1100) & (~no_dialysis_patient_drugs.code.isna())]

In [ ]:
print(mean, std, no_dialysis_patient_drugs_in_range.patient_id.nunique(), len(no_dialysis_patient_drugs_in_range))

225.86957994579944 299.6062771775065 19004 4165368


In [ ]:
#need both positive and negative samples
dialysis_patient_drugs_in_range['dialysis'] = 1
no_dialysis_patient_drugs_in_range['dialysis'] = 0
both = pd.concat([dialysis_patient_drugs_in_range[['patient_id', 'code_system', 'code', 'dialysis', 'brand']], no_dialysis_patient_drugs_in_range[['patient_id', 'code_system', 'code', 'dialysis', 'brand']]], ignore_index=True)
df = both
df["drug"] = df["code_system"].astype(str) + ":" + df["brand"]+":"+df["code"].astype(str)
df = df.drop_duplicates()

/tmp/ipython-input-3348213375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialysis_patient_drugs_in_range['dialysis'] = 1
/tmp/ipython-input-3348213375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dialysis_patient_drugs_in_range['dialysis'] = 0


In [ ]:
dialysis_map = df[["patient_id", "dialysis"]].drop_duplicates("patient_id")

In [ ]:
drug_matrix = pd.crosstab(df["patient_id"], df["drug"]).astype(int) #Each drug is a feature, 1 if the patient uses the drug, 0 if not
drug_matrix = drug_matrix.reset_index()  # brings 'patient_id' back as a colum

In [ ]:
#drug_patient = pd.merge(drugs, dialysis_patients, on='patient_id', how='inner') #drugs used by target patients

In [ ]:
features = drug_matrix.merge(dialysis_map, on='patient_id', how='inner')
features.shape

(23741, 26690)

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score

df = features

# df = pd.read_csv("your_data.csv") or merge your demographic + drug usage tables

# Drop patient_id
df = df.drop(columns=["patient_id"])

# Encode categorical variables
for col in df.select_dtypes(include="object"):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Define features and label
X = df.drop(columns=["dialysis"])
y = df["dialysis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_neg = (y_train == 0).sum()
num_pos = (y_train == 1).sum()
scale = num_neg / num_pos

model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    scale_pos_weight=scale,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Confusion matrix: [[TN, FP], [FN, TP]]
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
# Specificity
specificity = tn / (tn + fp)

print("Sensitivity (Recall):", sensitivity)
print("Specificity:", specificity)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))

In [ ]:
!pip install shap

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
import numpy as np
import pandas as pd

# shap_values can be a numpy array or a shap.Explanation object
# If it's a shap.Explanation (common in newer SHAP versions):
if hasattr(shap_values, "values"):
    shap_array = shap_values.values
else:
    shap_array = shap_values  # assume already array-like

# Take mean absolute SHAP value across all samples
mean_abs_shap = np.abs(shap_array).mean(axis=0)

# Get indices of top 10 features
top_idx = np.argsort(mean_abs_shap)[-10:][::-1]

# Get feature names (assuming X_train is a DataFrame)
top_features = X_train.columns[top_idx]

print("Top 10 features:")
print(top_features.tolist())

In [ ]:
shap.summary_plot(shap_values, X_train, max_display=10)
#Left = decreases prediction (e.g., not on dialysis)

#Right = increases prediction (e.g., likely on dialysis)

In [ ]:
import matplotlib.pyplot as plt

mean_abs_shap = pd.DataFrame(shap_values, columns=X.columns).abs().mean()
shap_importance = mean_abs_shap.sort_values(ascending=False)

xgb_importance = model.get_booster().get_score(importance_type='gain')
xgb_importance_df = pd.DataFrame.from_dict(xgb_importance, orient='index', columns=['gain']).sort_values(by='gain', ascending=False)

df_compare = pd.DataFrame({
    "shap": shap_importance,
    "xgb_gain": xgb_importance_df["gain"]
}).dropna()

df_compare.plot(kind='scatter', x='xgb_gain', y='shap', title='SHAP vs XGBoost Gain Importance')
plt.xlabel("XGBoost Gain")
plt.ylabel("Mean |SHAP value|")
plt.show()

#shap_importance
#Since SHAP values are in log-odds, here’s how to interpret 0.1:
#A feature with SHAP = ±0.1 changes the model’s log-odds by 0.1.
#That translates to a very small shift in predicted probability (e.g., from 0.50 → ~0.527).


In [ ]:
count = (shap_importance > 0).sum()
print(count)

In [ ]:
for topx in [100, 300, 600, 800, 1104]:
  # Get indices of top 10 features
  top_idx = np.argsort(mean_abs_shap)[-topx:][::-1]

  # Get feature names (assuming X_train is a DataFrame)
  top_features = X_train.iloc[:, top_idx]

  print("------------------------")
  print("Top features:", len(top_idx))
  model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    scale_pos_weight=scale,
    random_state=42
  )

  model.fit(top_features, y_train)

  test = X_test.iloc[:, top_idx]
  y_pred = model.predict(test)
  y_prob = model.predict_proba(test)[:, 1]

  # F1 score
  f1 = f1_score(y_test, y_pred)
  print("F1 Score:", f1)

  # Confusion matrix: [[TN, FP], [FN, TP]]
  cm = confusion_matrix(y_test, y_pred)
  tn, fp, fn, tp = cm.ravel()

  # Sensitivity (Recall)
  sensitivity = tp / (tp + fn)
  # Specificity
  specificity = tn / (tn + fp)

  print("Sensitivity (Recall):", sensitivity)
  print("Specificity:", specificity)

  print("Accuracy:", accuracy_score(y_test, y_pred))
  print("AUC:", roc_auc_score(y_test, y_prob))

In [ ]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    f1_score, confusion_matrix, accuracy_score, roc_auc_score
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    AdaBoostClassifier,
    BaggingClassifier
)
from xgboost import XGBRFClassifier  # optional: XGBoost's random-forest variant

# ---------------------------------
# Use your existing dataframe `features`
# ---------------------------------
df = features

# If reading from CSV or merging:
# df = pd.read_csv("your_data.csv") or merge your demographic + drug usage tables

# Drop patient_id
df = df.drop(columns=["patient_id"])

# Encode categorical variables (same as your code)
for col in df.select_dtypes(include="object"):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Define features and label
X = df.drop(columns=["dialysis"])
y = df["dialysis"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ---------------------------------
# Helper to evaluate and print in the same format
# ---------------------------------
def evaluate_and_print(model, name=None):
    if name is None:
        name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Use predicted probabilities for AUC when available; fall back to decision_function
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        # Some models output raw scores
        y_prob = model.decision_function(X_test)
    else:
        # If neither is available, use predictions as scores (not ideal, but keeps format)
        y_prob = y_pred

    # F1
    f1 = f1_score(y_test, y_pred)
    print(f"{name}")
    print("F1 Score:", f1)

    # Confusion matrix: [[TN, FP], [FN, TP]]
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    # Sensitivity (Recall) and Specificity
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    print("Sensitivity (Recall):", sensitivity)
    print("Specificity:", specificity)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("AUC:", roc_auc_score(y_test, y_prob))
    print("-" * 60)


# ---------------------------------
# Tree-based models (excluding the original XGBClassifier)
# ---------------------------------
models = [
    # Single tree
    DecisionTreeClassifier(random_state=42),

    # Random forests & variants
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    XGBRFClassifier(  # XGBoost's RF-style trees (uses subsampling for RF behavior)
        n_estimators=300,
        subsample=0.8,
        colsample_bynode=0.8,
        random_state=42,
        eval_metric="logloss",
        n_jobs=-1
    ),

    # Boosted trees
    GradientBoostingClassifier(random_state=42),
    HistGradientBoostingClassifier(random_state=42),
]

for m in models:
    evaluate_and_print(m)

In [ ]:
!pip install requests

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=["patient_id", "dialysis"])
df_no_dup = df.drop_duplicates()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df_no_dup = pd.read_csv("/content/drive/MyDrive/df_no_dup.csv")

In [ ]:
df_no_dup.drug.nunique()

26675

In [ ]:
import argparse
import csv
import time
from typing import Dict, List, Optional, Tuple
import requests

RXNAV = "https://rxnav.nlm.nih.gov/REST"

def backoff_sleep(attempt: int) -> None:
    # simple exponential backoff: 0.5, 1, 2, 4, 8 (cap at ~8s)
    delay = min(0.5 * (2 ** attempt), 8.0)
    time.sleep(delay)

def http_get_json(url: str, params: Optional[Dict[str, str]] = None, retries: int = 4) -> Optional[Dict]:
    for attempt in range(retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=15)
            if resp.status_code == 200:
                return resp.json()
            # Retry on 5xx / rate-limit
            if resp.status_code >= 500 or resp.status_code == 429:
                backoff_sleep(attempt)
                continue
            # Other errors -> no retry
            return None
        except requests.RequestException:
            backoff_sleep(attempt)
    return None

def ndc_to_rxcui(ndc: str) -> Optional[str]:
    """Resolve an NDC code to an RxCUI."""
    # RxNav accepts formats with or without dashes. We'll pass as-is first.
    data = http_get_json(f"{RXNAV}/rxcui.json", params={"idtype": "ndc", "id": ndc})
    if not data:
        return None
    id_group = data.get("idGroup", {})
    rxcui = id_group.get("rxnormId")
    if isinstance(rxcui, list) and rxcui:
        return rxcui[0]
    if isinstance(rxcui, str) and rxcui.strip():
        return rxcui.strip()
    return None

def get_properties(rxcui: str) -> Tuple[Optional[str], Optional[str]]:
    """Return (name, tty) for an RxCUI."""
    data = http_get_json(f"{RXNAV}/rxcui/{rxcui}/properties.json")
    if not data:
        return None, None
    props = data.get("properties", {})
    return props.get("name"), props.get("tty")

def get_related_names(rxcui: str, ttys: List[str]) -> List[str]:
    """Return a list of related names filtered by the given TTYs (e.g., ['BN'] for brand names)."""
    tty_param = ",".join(ttys)
    data = http_get_json(f"{RXNAV}/rxcui/{rxcui}/related.json", params={"tty": tty_param})
    results: List[str] = []
    if not data:
        return results
    groups = data.get("relatedGroup", {}).get("conceptGroup", [])
    for g in groups:
        if not g or "conceptProperties" not in g:
            continue
        for cp in g["conceptProperties"]:
            name = cp.get("name")
            if name and name not in results:
                results.append(name)
    return results

def derive_generic_and_brand(rxcui: str) -> Tuple[Optional[str], List[str]]:
    """
    Heuristic derivation:
      - GenericName: prefer IN (ingredient) from related; else use SCD (clinical drug);
        else fall back to the RxCUI's own name if it isn't a pure brand (BN/SBD).
      - BrandNames: collect BN + SBD related names.
    """
    # Get properties to know the base name + tty
    base_name, base_tty = get_properties(rxcui)

    # Get brand names via BN and SBD
    brand_names = get_related_names(rxcui, ["BN", "SBD"])

    # Try to get ingredient/generic names via IN first
    ing_names = get_related_names(rxcui, ["IN"])
    if ing_names:
        generic_name = ing_names[0]
        return generic_name, brand_names

    # Fall back to SCD (Semantic Clinical Drug) or GPCK/SCDF if available
    scd_like = get_related_names(rxcui, ["SCD", "SCDF", "GPCK"])
    if scd_like:
        return scd_like[0], brand_names

    # Last resort, use the base name if tty is not a brand
    if base_name and base_tty and base_tty not in {"BN", "SBD"}:
        return base_name, brand_names

    # If still nothing, just return base_name as generic with empty brand list
    return base_name, brand_names

def resolve_row(code_system: str, brand_field: str, code: str) -> Tuple[str, str, str, Optional[str], Optional[str], str]:
    """
    Return a tuple for output row:
      CodeSystem, BrandField, Code, RxCUI, GenericName, BrandNames(semi-colon)
    """
    notes = ""
    rxcui: Optional[str] = None

    if code_system == "RxNorm":
        rxcui = code.strip()
    elif code_system == "NDC":
        rxcui = ndc_to_rxcui(code.strip())
        if not rxcui:
            notes = "NDC→RxCUI resolution failed"
    else:
        notes = f"Unsupported CodeSystem: {code_system}"

    generic_name: Optional[str] = None
    brand_names: List[str] = []

    if rxcui:
        generic_name, brand_names = derive_generic_and_brand(rxcui)
        if not generic_name and not brand_names:
            notes = (notes + "; " if notes else "") + "No names found from RxNav"

    brand_names_str = "; ".join(brand_names) if brand_names else ""

    return code_system, brand_field, code, (rxcui or ""), (generic_name or ""), brand_names_str, notes

#in_path="drug_codes.csv"
out_path="/content/drive/MyDrive/"
sleep = 0.05

#df = pd.read_csv("drug_codes.csv")
#df = drug_list

header_out = ["CodeSystem", "BrandField", "Code", "RxCUI", "GenericName", "BrandNames", "Notes"]
out = pd.DataFrame(columns=header_out)

#for i, r in df_no_dup.iterrows():
for i, r in enumerate(df_no_dup.iterrows(), start=1):
        if i<= 26600:
          continue
        cs = (r[1].get("code_system") or "").strip()
        bf = (r[1].get("brand") or "").strip()
        code = (r[1].get("code") or "").strip()

        out_row = resolve_row(cs, bf, code)
        out.loc[len(out)] = out_row

        #time.sleep(sleep)

        if i> 0 and i % 200 == 0:
              print(f"Processed {i} rows...")
              print(len(out))
              out.to_csv(f"{out_path}/out_{i}.csv")

#out.to_csv("out.csv")

In [ ]:
out.head()

In [ ]:
df_no_dup.to_csv("/content/drive/MyDrive/df_no_dup.csv")

In [ ]:
df_no_dup.columns

Index(['Unnamed: 0', 'code_system', 'code', 'brand', 'drug'], dtype='object')

In [ ]:
%cd /content/drive/MyDrive/simons

/content/drive/MyDrive/simons


In [ ]:
import pandas as pd
from functools import lru_cache

# ========= CONFIG =========
# Which indication relations to accept
RELA_ALLOW = {"may_treat", "treats", "has_indication"}  # case-insensitive match

# Disease TUI whitelist (expand if you want broader "conditions")
DISEASE_TUIS = {"T047", "T191"}  # Disease or Syndrome, Neoplastic Process

# Keyword screen for heart/kidney
HEART_KIDNEY_KEYWORDS = ("heart", "cardio", "coronary", "myocard", "angina",
                   "atrial", "ventric", "arrhythm", "kidney", "renal", "nephro",
                         "glomerul", "dialysis",
                   "ckd", "aki", "nephritis", "proteinuria")

# Map incoming code_system values -> UMLS SAB to search in MRCONSO
SAB_MAP = {
    "rxnorm": "RXNORM",
    "ndc": "RXNORM",   # NDCs are present in the RXNORM source in UMLS
    "atc": "ATC",
    # add more if your data includes them (e.g., "snomedct_us": "SNOMEDCT_US")
}

# ========= LOAD UMLS RRF (minimal columns only) =========
# MRCONSO: CUI,SAB,CODE,STR + also TTY for better brand/name handling
mrconso = pd.read_csv(
    "MRCONSO.RRF", sep="|", header=None, dtype=str,
    usecols=[0, 11, 13, 14, 12]
)
mrconso.columns = ["CUI", "SAB", "CODE", "STR", "TTY"]
print(f"{len(mrconso)} mrconso")
# MRREL: CUI1, CUI2, RELA (plus SAB for provenance)
mrrel = pd.read_csv(
    "MRREL.RRF", sep="|", header=None, dtype=str,
    usecols=[0, 4, 7, 10]
)
mrrel.columns = ["CUI1", "CUI2", "RELA", "SAB"]
print(f"{len(mrrel)} mrrel")
# MRSTY: CUI,TUI
mrsty = pd.read_csv(
    "MRSTY.RRF", sep="|", header=None, dtype=str,
    usecols=[0, 1]
)
mrsty.columns = ["CUI", "TUI"]
print(f"{len(mrsty)}, mrsty")
# ========= LIGHTWEIGHT INDICES =========
# 1) (SAB, CODE) -> set(CUI)  (for fast code->CUI lookup)
code_index = {}
for sab, code, cui in mrconso[["SAB", "CODE", "CUI"]].dropna().itertuples(index=False):
    key = (sab, code)
    code_index.setdefault(key, set()).add(cui)

print("1) done")
# 2) CUI -> preferred string (pick RXNORM or any; fallback to first)
#    This is just for readable outputs.
name_pref_order = ["RXNORM", "SNOMEDCT_US", "MSH", "ICD10CM"]
cui_name = {}

# prefer TTYs that look like ingredients/brands for drugs:
preferred_ttys = {"IN", "PIN", "SCD", "SBD", "BN", "BPCK", "MIN", "SCDC", "SBDC"}

def choose_name(df):
    # prefer preferred sources, then preferred TTYs, then first
    for sab in name_pref_order:
        sub = df.loc[df["SAB"] == sab]
        if not sub.empty:
            sub2 = sub.loc[sub["TTY"].isin(preferred_ttys)]
            if not sub2.empty:
                return sub2.iloc[0]["STR"]
            return sub.iloc[0]["STR"]
    sub2 = df.loc[df["TTY"].isin(preferred_ttys)]
    if not sub2.empty:
        return sub2.iloc[0]["STR"]
    return df.iloc[0]["STR"]

for cui, group in mrconso.groupby("CUI", sort=False):
    try:
        cui_name[cui] = choose_name(group)
    except Exception:
        cui_name[cui] = group.iloc[0]["STR"]

print("2) done")
# 3) Disease filter set: CUIs that are diseases per MRSTY
disease_cuis = set(mrsty.loc[mrsty["TUI"].isin(DISEASE_TUIS), "CUI"])

# 4) Relations index: from drug CUI -> set of target CUIs that match indication RELAs
#    (We’ll filter RELA by allowed set, case-insensitive)
mrrel["RELA_lc"] = mrrel["RELA"].str.lower()
rel_mask = mrrel["RELA_lc"].isin({r.lower() for r in RELA_ALLOW})
drug_to_targets = {}
for cui1, cui2 in mrrel.loc[rel_mask, ["CUI1", "CUI2"]].itertuples(index=False):
    if pd.isna(cui1) or pd.isna(cui2):
        continue
    drug_to_targets.setdefault(cui1, set()).add(cui2)

print("3) done")
# ========= UTILITIES =========
def norm_code_system(s: str) -> str:
    return (s or "").strip().lower()

def norm_code_val(code_system: str, code: str) -> str:
    """Normalize codes a bit (e.g., strip hyphens for NDC)."""
    v = (code or "").strip()
    if code_system == "ndc":
        return v.replace("-", "")
    return v

def candidate_sabs(code_system: str):
    """Return list of SABs to try for this code system (with fallbacks)."""
    cs = norm_code_system(code_system)
    if cs in SAB_MAP:
        # for NDC, try both raw NDC (no dashes) and with dashes since RXNORM stores both styles
        if cs == "ndc":
            return ["RXNORM"]
        return [SAB_MAP[cs]]
    # unknown -> try the most likely sources
    return ["RXNORM", "ATC", "SNOMEDCT_US"]

@lru_cache(maxsize=100_000)
def code_to_cuis(code_system: str, code: str):
    """Resolve incoming (code_system, code) to CUIs using MRCONSO only."""
    cs = norm_code_system(code_system)
    raw = (code or "").strip()
    normed = norm_code_val(cs, raw)
    cuis = set()

    for sab in candidate_sabs(cs):
        # exact code
        cuis |= code_index.get((sab, raw), set())
        # normalized code (e.g., NDC without hyphens)
        if normed != raw:
            cuis |= code_index.get((sab, normed), set())

    return cuis

@lru_cache(maxsize=100_000)
def brand_to_cuis(brand: str):
    """Very light fallback: match brand name to CUIs in RXNORM/brand-ish TTYs."""
    if not brand or not brand.strip():
        return set()
    b = brand.strip().lower()
    sub = mrconso.loc[
        (mrconso["SAB"] == "RXNORM") &
        (mrconso["TTY"].isin({"BN", "SBD", "SBDC"})) &
        (mrconso["STR"].str.lower() == b),
        "CUI"
    ]
    return set(sub.unique())

@lru_cache(maxsize=200_000)
def treated_disease_cuis(drug_cui: str):
    """From a drug CUI, fetch indication/‘treats’ target CUIs filtered to diseases."""
    targets = drug_to_targets.get(drug_cui, set())
    return {c for c in targets if c in disease_cuis}

def diseases_for_code(code_system: str, code: str, brand: str = None):
    """Main: resolve to CUIs, follow indication relations to disease CUIs, return names."""
    cuis = set(code_to_cuis(code_system, code))
    if not cuis and brand:
        cuis |= brand_to_cuis(brand)

    if not cuis:
        return []

    out = set()
    for dcui in cuis:
        for tcui in treated_disease_cuis(dcui):
            out.add((tcui, cui_name.get(tcui, tcui)))
    # Return sorted by name for stability
    return sorted(out, key=lambda x: x[1].lower())

def is_heart_or_kidney(disease_names):
    text = " ".join(name for _, name in disease_names).lower()
    return any(k in text for k in HEART_KIDNEY_KEYWORDS)

# ========= APPLY TO YOUR DATAFRAME =========
# Your input df should have columns: code_system, code, brand
# Example:
# df = pd.DataFrame([
#     {"code_system": "RxNorm", "code": "1049630", "brand": "Lisinopril"},
#     {"code_system": "NDC", "code": "00071-1013", "brand": "Lasix"},
#     {"code_system": "RxNorm", "code": "705129", "brand": "Atorvastatin"},
# ])

def annotate_df(df):
    results = []
    for row in df.itertuples(index=False):
        idx = row.index
        cs = getattr(row, "code_system")
        code = getattr(row, "code")
        brand = getattr(row, "brand", None)
        dz = diseases_for_code(cs, code, brand)
        results.append({
            "treats_heart_or_kidney": is_heart_or_kidney(dz),
            "treated_diseases": [name for _, name in dz],  # optional: keep full list
        })
        if idx >0 and idx % 500 == 0:
          print(f"{row} records done")
          x = pd.concat([df.reset_index(drop=True), pd.DataFrame(results)], axis=1)
          x.to_csv(f"out_{idx}.csv")
    return pd.concat([df.reset_index(drop=True), pd.DataFrame(results)], axis=1)

# Usage:
#df = pd.read_csv("my_drug_list.csv")  # columns: code_system, code, brand
df_out = annotate_df(drugs)
df_out.to_csv("drug_treatment_results.csv", index=False)


17139786 mrconso
62908136 mrrel
3798191, mrsty
1) done


KeyboardInterrupt: 

In [ ]:
import pandas as pd

drug_treatment_results = pd.read_csv("drug_treatment_results.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'drug_treatment_results.csv'

In [ ]:
import requests
from urllib.parse import quote

# ── CONFIG ─────────────────────────────────────────────────────────────────────
UMLS_APIKEY = "YOUR_UMLS_API_KEY_HERE"  # ← put your key here
UMLS_BASE = "https://uts-ws.nlm.nih.gov/rest"
CAS_TGT_URL = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
SERVICE = "http://umlsks.nlm.nih.gov"   # constant per UTS docs

# Indication predicate labels you’ll commonly see (MED-RT/NDF-RT/SNOMED flavors)
INDICATION_LABELS = {
    "may_treat", "has_indication", "has_accepted_indication",
    "may_prevent", "has_therapeutic_class"  # keep first two as primary
}

# Simple keyword heuristics to tag diseases as heart/kidney related.
HEART_KEYWORDS  = {"heart", "cardio", "coronary", "myocard", "angina",
                   "atrial", "ventric", "hypertension", "blood pressure", "arrhythm"}
KIDNEY_KEYWORDS = {"kidney", "renal", "nephro", "glomerul", "dialysis",
                   "ckd", "aki", "nephritis", "proteinuria"}

# ── AUTH HELPERS ───────────────────────────────────────────────────────────────
def get_tgt(apikey: str) -> str:
    r = requests.post(CAS_TGT_URL, data={"apikey": apikey})
    r.raise_for_status()
    # TGT URL is in the HTML form action attribute
    import re
    m = re.search(r'action="([^"]+)"', r.text)
    if not m:
        raise RuntimeError("Could not parse TGT from CAS response")
    return m.group(1)

def get_service_ticket(tgt_url: str) -> str:
    r = requests.post(tgt_url, data={"service": SERVICE})
    r.raise_for_status()
    return r.text

def uts_get(path: str, params=None, tgt_url=None):
    params = params or {}
    # get one-time service ticket for each call
    st = get_service_ticket(tgt_url)
    params["ticket"] = st
    url = f"{UMLS_BASE}{path}"
    r = requests.get(url, params=params, timeout=20)
    r.raise_for_status()
    return r.json()

# ── LOOKUPS ────────────────────────────────────────────────────────────────────
def rxcui_to_cui(rxcui: str, tgt_url: str) -> str | None:
    # RxNorm → UMLS CUI via UTS content endpoint
    data = uts_get(f"/content/current/source/RXNORM/{quote(rxcui)}", tgt_url=tgt_url)
    return data.get("result", {}).get("ui")  # often equals the RxCUI for RxNorm atoms; fallback below

def normalize_to_cui(identifier: str, tgt_url: str) -> str | None:
    # Heuristic: if it’s digits and relatively small → likely RxCUI; try both ways.
    if identifier.isdigit():
        # 1) Try treating as RxCUI
        try:
            cui = rxcui_to_cui(identifier, tgt_url)
            if cui and cui.startswith("C"):
                return cui
        except Exception:
            pass
    # 2) Try treating as UMLS CUI directly (C#######)
    if identifier.upper().startswith("C"):
        # validate it exists
        try:
            _ = uts_get(f"/content/current/CUI/{identifier}", tgt_url=tgt_url)
            return identifier.upper()
        except Exception:
            pass
    # 3) As a last resort, search Metathesaurus by string
    try:
        data = uts_get("/search/current", params={"string": identifier}, tgt_url=tgt_url)
        results = data.get("result", {}).get("results", [])
        for item in results:
            if item.get("ui", "").startswith("C"):
                return item["ui"]
    except Exception:
        pass
    return None

def get_indications(cui: str, tgt_url: str):
    """Return list of (disease_cui, disease_name, relationLabel, source) pairs for indications."""
    out = []
    # Traverse relations from the drug CUI
    data = uts_get(f"/content/current/CUI/{cui}/relations", tgt_url=tgt_url)
    for rel in data.get("result", []):
        label = (rel.get("relationLabel") or "").lower().replace(" ", "_")
        if label not in INDICATION_LABELS:
            continue
        related = rel.get("relatedId")
        name = rel.get("relatedIdName") or ""
        src  = rel.get("rootSource") or ""
        # Only consider disease/condition-ish targets; filter a bit by name
        if not related or not related.startswith("C"):
            continue
        out.append((related, name, label, src))
    return out

def tag_system(name: str) -> set[str]:
    s = name.lower()
    tags = set()
    if any(k in s for k in HEART_KEYWORDS):
        tags.add("heart")
    if any(k in s for k in KIDNEY_KEYWORDS):
        tags.add("kidney")
    return tags

def annotate_indications(indications: list[tuple[str, str, str, str]], tgt_url: str):
    """Add simple heart/kidney tags; you can enrich by fetching semantic types if desired."""
    results = []
    for dcui, dname, label, src in indications:
        tags = tag_system(dname)
        results.append({
            "disease_cui": dcui,
            "disease_name": dname,
            "relation": label,
            "source": src,
            "tags": sorted(tags)
        })
    return results

# ── PUBLIC ENTRYPOINT ──────────────────────────────────────────────────────────
def drug_indications_heart_kidney(identifier: str):
    """
    identifier: RxCUI (e.g., '8610') or UMLS CUI (e.g., 'C0020615') or even a drug string.
    Returns a dict with indications + tags.
    """
    tgt_url = get_tgt(UMLS_APIKEY)
    cui = normalize_to_cui(identifier, tgt_url)
    if not cui:
        return {"input": identifier, "error": "Could not resolve to a UMLS CUI"}

    inds = get_indications(cui, tgt_url)
    annotated = annotate_indications(inds, tgt_url)
    # convenience booleans
    treats_heart  = any("heart"  in r["tags"] for r in annotated)
    treats_kidney = any("kidney" in r["tags"] for r in annotated)
    return {
        "input": identifier,
        "cui": cui,
        "treats_heart": treats_heart,
        "treats_kidney": treats_kidney,
        "indications": annotated
    }

# ── EXAMPLE ────────────────────────────────────────────────────────────────────
# print(drug_indications_heart_kidney("8610"))  # e.g., lisinopril RxCUI
# print(drug_indications_heart_kidney("lisinopril"))